# Examples for loading fmriprep results

This notebook provides some examples for creating BData from outputs of [fmriprep](https://fmriprep.readthedocs.io/en/stable/usage.html).

- fmriprep >= 1.2 is recommended
- fmriprep should produce data in the following four spaces (this requirement will be relaxed in the future update):
    - Volume in the subject anatomical reference space ('T1w')
    - Volume in the standard space ('template')
    - Surface on the subject individual surface ('fsnative')
    - Surface on fsaverage ('fsaverage')


Currently, the example data are only available for Kamitani Lab members due to privacy policy. We're going to preprare example data from public datasets.

To Kamitani Lab members: you can find the example data at `/home/aoki/work/bdpy/examples/data`.

In [ ]:
import bdpy
from bdpy.mri.fmriprep import create_bdata_fmriprep

## Create BData from fmriprep results

In [ ]:
help(create_bdata_fmriprep)

`create_bdata_fmriprep` takes three arguments (one positional and two keyword args).
    
- Path to the BIDS root directory.
- `data_mode`: the data to be extracted from fmriprep outputs ('volume_native', 'volume_standard', 'surface_native', or 'surface_standard').
- `label_mapper`: a dictionary specifying tables to convert non-numerical values in task event files into numbers.

In [ ]:
# Volume, subject native space
dat_vol_native = create_bdata_fmriprep('data/fmriprep-1.2/SB181112/bids/',
                                       data_mode='volume_native',
                                       label_mapper={'stimulus_name' : 'data/stim_test.tsv'})

In [ ]:
# Volume, standard space (MNI)
dat_vol_standard = create_bdata_fmriprep('data/fmriprep-1.2/SB181112/bids/',
                                         data_mode='volume_standard',
                                         label_mapper={'stimulus_name' : 'data/stim_test.tsv'})

In [ ]:
# Surface on subject native
dat_surf_native = create_bdata_fmriprep('data/fmriprep-1.2/SB181112/bids/',
                                        data_mode='surface_native',
                                        label_mapper={'stimulus_name' : 'data/stim_test.tsv'})

In [ ]:
# Surface on fsaverage
dat_surf_standard = create_bdata_fmriprep('data/fmriprep-1.2/SB181112/bids/',
                                         data_mode='surface_standard',
                                         label_mapper={'stimulus_name' : 'data/stim_test.tsv'})

## Playground

In [ ]:
# Check data size
print(dat_vol_native.dataset.shape)
#print(dat_vol_standard.dataset.shape)
print(dat_surf_native.dataset.shape)
#print(dat_surf_standard.dataset.shape)

In [ ]:
# Get left surface (vertices)
data = dat_surf_native.get('VertexLeft')
data.shape
#dat_surf_native.get_metadata('vertex_index', where='VertexLeft')

## Adding ROIs

In [ ]:
# Volume data
#from bdpy.mri import add_roimask

#TBA

In [ ]:
# Individual surface

from bdpy.mri import add_roilabel

surf_label_dir = './data/anatomy/SB181112/nifti/roi/roi_label/hcp180'

label_files = [(os.path.join(surf_label_dir, 'lh.L_V1_ROI.label'), 'left'),
               (os.path.join(surf_label_dir, 'rh.R_V1_ROI.label'), 'right')]

for label in label_files:
    label_file = label[0]  # Label file
    label_hemi = label[1]  # Left or right

    if label_hemi == 'left':
        vertex_data = 'VertexLeft'
    elif label_hemi == 'right':
        vertex_data = 'VertexRight'
    else:
        raise ValueError('Unknown hemi: %s' % label_hemi)
    
    bdata = add_roilabel(dat_surf_native, label_file, vertex_data, prefix='hcp180', verbose=True)

In [ ]:
dat_surf_native.get('hcp180_rh.R_V1_ROI').shape

In [ ]:
# Standard surface (fsaverage)

from bdpy.mri import add_roilabel
import nibabel.freesurfer

fs_subjects_dir = os.environ['SUBJECTS_DIR']  # FreeSurfer subjects directory

surf_label_dir = os.path.join(fs_subjects_dir, 'fsaverage', 'label')  # Take labels from fsaverage

label_files = [(os.path.join(surf_label_dir, 'lh.aparc.a2009s.annot'), 'left'),
               (os.path.join(surf_label_dir, 'rh.aparc.a2009s.annot'), 'right')] # Destrieux atlas

for label in label_files:
    label_file = label[0]  # Label file
    label_hemi = label[1]  # Left or right

    if label_hemi == 'left':
        vertex_data = 'VertexLeft'
    elif label_hemi == 'right':
        vertex_data = 'VertexRight'
    else:
        raise ValueError('Unknown hemi: %s' % label_hemi)

    bdata = add_roilabel(dat_surf_standard, label_file, vertex_data, prefix='freesurfer_destrieux', verbose=True)

In [ ]:
bdata.get('freesurfer_destrieux_lh.G_and_S_frontomargin').shape